# Satellite Imagery Downloader - Daily

In [1]:
import requests
import json
import numpy as np
from zipfile import ZipFile
from PIL import Image
import utm
import time
import cv2
import scipy.sparse
import sys
import glob
import os
import io
import shutil
from matplotlib import pyplot as plt

### Functions

In [2]:
def download_file(url, target_path, creds):
    response = requests.get(url, stream=True, auth = creds);
    handle = open(target_path, "wb");
    for chunk in response.iter_content(chunk_size=512):
        if chunk:  # filter out keep-alive new chunks
            handle.write(chunk);
    handle.close();

In [3]:
def getImageFromZip(file_name, suffix="_B08.jp2"):
    arr = np.empty(0);
    try:

        with ZipFile(file_name, 'r') as zip:
            for file_or_dir in zip.filelist:
                if (suffix in file_or_dir.filename):
                    fn = file_or_dir.filename;
                    data = zip.read(fn);
                    arr = np.array(Image.open(io.BytesIO(data)));
    except:
        print("Error while reading file");
    return arr;

In [4]:
def getMaskedImage(img, mask):
    arr = cv2.bitwise_and(img, img, mask=mask.toarray());
    return arr;

In [5]:
def hist_match(source, template):
    """
    source: https://stackoverflow.com/questions/32655686/histogram-matching-of-two-images-in-python-2-x
    Adjust the pixel values of a grayscale image such that its histogram
    matches that of a target image

    Arguments:
    -----------
        source: np.ndarray
            Image to transform; the histogram is computed over the flattened
            array
        template: np.ndarray
            Template image; can have different dimensions to source
    Returns:
    -----------
        matched: np.ndarray
            The transformed output image
    """

    oldshape = source.shape
    source = source.ravel()
    template = template.ravel()

    # get the set of unique pixel values and their corresponding indices and
    # counts
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    t_values, t_counts = np.unique(template, return_counts=True)

    # take the cumsum of the counts and normalize by the number of pixels to
    # get the empirical cumulative distribution functions for the source and
    # template images (maps pixel value --> quantile)
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # interpolate linearly to find the pixel values in the template image
    # that correspond most closely to the quantiles in the source image
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)

### Processing

1. Download latest product (zip)
1. Get Mask image
1. Extract ...B08.jp2 image from zip
1. Mask B08 image by using mask
1. Outlier removal by using base line information
1. Get last image for comparison
1. Save this new image as latest image
1. Move zip file to history
1. Compare new and last image
1. Build heatmap image
1. Get points from highests and lowests values
1. Save information as geojson



http://colorbrewer2.org/#type=diverging&scheme=PRGn&n=3

#### Download Latest Products

In [ ]:
creds = ('<username>', '<password>')
query1 = 'https://scihub.copernicus.eu/dhus/search?q=S2A_MSIL1C_* AND ( footprint:"Intersects(POLYGON((25.602013093645457 65.04929472186046,26.55684592774098 65.04929472186046,26.55684592774098 65.6153691935298,25.602013093645457 65.6153691935298,25.602013093645457 65.04929472186046)))" )&rows=10&start=0&format=json';

In [ ]:
response = requests.get(query1, auth=creds)
data1 = response.json()

In [ ]:
links = {}
for entry in data1['feed']['entry']:
    links[entry['title'] + '.zip'] = entry['link'][0]['href'];

In [ ]:
for l in links:
    if (not os.path.exists(os.path.join("./history", l))):
        print("New product:", l);
        download_file(links[l], "./daily/" + l, creds);
        file_name = "./daily/" + l
        hist_name = "./history/" + l

In [6]:
file_name = "./daily/S2A_MSIL1C_20191005T095031_N0208_R079_T35WMN_20191005T102509.zip" 
hist_name = "./history/S2A_MSIL1C_20191005T095031_N0208_R079_T35WMN_20191005T102509.zip" 

#### Get Mask Image

In [7]:
sp_mask = scipy.sparse.load_npz('powerline_mask.npz');
final = np.load("powerline_elevations_dist.npy")

#### Extract ...B08.jp2 image from zip

In [8]:
img_b08 = getImageFromZip(file_name)

/home/mlahdeaho/anaconda3/envs/cudf/lib/python3.6/site-packages/PIL/Image.py:2731: DecompressionBombWarning: Image size (120560400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


#### Mask B08 image by using mask

In [9]:
if (len(img_b08) > 0):
    masked = getMaskedImage(img_b08, sp_mask);
    masked = masked.astype('float64')
    masked[masked == 0] = np.nan

#### Outlier removal by using base line information

In [10]:
q1 = final[:,:,0]
q1_max = np.nanmax(q1);

In [11]:
masked[masked > q1_max] = np.nan

/home/mlahdeaho/anaconda3/envs/cudf/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


#### Get last image for comparison


In [12]:
img_last = np.load("powerline_elevations_last.npy")

#### Compare new and last image

In [13]:
A = hist_match(masked, q1)

In [14]:
B = hist_match(img_last, q1)

In [15]:
diff = np.subtract(B, A)

#### Build heatmap image

In [57]:
heatmap = np.empty((10980,10980,4), dtype=np.float64)
heatmap[:,:,0] = diff
heatmap[:,:,1] = diff
heatmap[:,:,2] = diff
heatmap[:,:,3] = diff

In [58]:
heatmap[:,:,0][heatmap[:,:,0] < 1] = 0
heatmap[:,:,0][(heatmap[:,:,0] > 0) & (heatmap[:,:,0] < 501)] = 85
heatmap[:,:,0][(heatmap[:,:,0]  > 500) & (heatmap[:,:,0] < 1001)] = 170
heatmap[:,:,0][heatmap[:,:,0] > 1000] = 255

In [59]:
heatmap[:,:,1][heatmap[:,:,1] != np.nan] = 0

In [60]:
heatmap[:,:,2][heatmap[:,:,2] > 0] = 0
heatmap[:,:,2][(heatmap[:,:,2] > -500) & (heatmap[:,:,2] < 1)] = 85
heatmap[:,:,2][(heatmap[:,:,2] > -1000) & (heatmap[:,:,2] < -501)] = 170
heatmap[:,:,2][heatmap[:,:,2] < -1001] = 255

In [61]:
heatmap[:,:,3][heatmap[:,:,3] == 0] = np.nan
heatmap[:,:,3][~np.isnan(heatmap[:,:,3])] = 255
heatmap[:,:,3][np.isnan(heatmap[:,:,3])] = 0

In [62]:
cv2.imwrite("heatmap_rgba.png", heatmap.astype('uint8'))

True

#### Get points from highests and lowests values

#### Save information as geojson

#### Save this new image as latest image

In [ ]:
np.save("powerline_elevations_last.npy", masked)
np.save("powerline_images/powerline_%s.npy" % (file_name[46:-4]), masked)

#### Move zip to history

In [ ]:
shutil.move(file_name, hist_name);

### Visualize

In [ ]:
np.nanmax(diff)

In [ ]:
h1 = np.histogram(A[~np.isnan(A)], bins=[0,1000,2000,3000,4000,5000,6000,7000,32000])

In [ ]:
h2 = np.histogram(B[~np.isnan(B)], bins=[0,1000,2000,3000,4000,5000,6000,7000,32000])

In [ ]:
h3 = np.histogram(diff[~np.isnan(diff)], bins=[0,1000,2000,3000,4000,5000,6000,7000,32000])

In [ ]:
plt.plot(h1[1][:-1], h1[0])
plt.plot(h2[1][:-1], h2[0])
plt.plot(h3[1][:-1], h3[0])
plt.show()

In [ ]:
len(diff[abs(diff) > 500])

In [22]:
len(diff[(diff > -500) & (diff < 1)])

/home/mlahdeaho/anaconda3/envs/cudf/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.
/home/mlahdeaho/anaconda3/envs/cudf/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.


110333

In [63]:
aimg = cv2.imread("heatmap_rgba.png", cv2.IMREAD_UNCHANGED)

In [64]:
aimg[:,:,3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)